In [72]:
import sys
sys.path.append("..")

from functions import get_data, data_split_train_val

import torch
import torch.nn as nn
import torch.optim as optim

In [73]:
# Load Fashion-MNIST CSV (test set)
X, y = get_data("../data/fashion-mnist_test.csv")

X_train, X_val, y_train, y_val = data_split_train_val(X, y)

print("Train:", X_train.shape)
print("Val:", X_val.shape)

Train: (8000, 784)
Val: (2000, 784)


In [74]:
# convert numpy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).long()

X_val_tensor = torch.from_numpy(X_val).float()
y_val_tensor = torch.from_numpy(y_val).long()

In [75]:
print("Before reshape:")
print("X_train_tensor shape:", X_train_tensor.shape)
print("X_val_tensor shape:", X_val_tensor.shape)

Before reshape:
X_train_tensor shape: torch.Size([8000, 784])
X_val_tensor shape: torch.Size([2000, 784])


In [76]:
num_train = X_train_tensor.shape[0]
num_val = X_val_tensor.shape[0]

print(num_train, num_val)

8000 2000


In [77]:
X_train_tensor = X_train_tensor.reshape(num_train, 1, 28, 28)
X_val_tensor = X_val_tensor.reshape(num_val, 1, 28, 28)
print("X_train_tensor reshape:", X_train_tensor.shape)
print("X_val_tensor reshape:", X_val_tensor.shape)

X_train_tensor reshape: torch.Size([8000, 1, 28, 28])
X_val_tensor reshape: torch.Size([2000, 1, 28, 28])


In [78]:
class CNN_ReLU(nn.Module):
    def __init__(self):
        super().__init__()

        self.activation = nn.ReLU(0.01)

        self.conv1 = nn.Conv2d(1, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)

        self.fc1 = nn.Linear(16 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = self.pool(x)

        x = self.activation(self.conv2(x))
        x = self.pool(x)

        x = torch.flatten(x, 1)

        x = self.activation(self.fc1(x))
        return self.fc2(x)

In [81]:
data = CNN_ReLU()   # create the model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(data.parameters(), lr=0.01)

epochs = 20

for epoch in range(epochs):
    optimizer.zero_grad()
    
    outputs = data(X_train_tensor)         
    loss = criterion(outputs, y_train_tensor)
    
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 1/20, Loss: 2.3036
Epoch 2/20, Loss: 2.2438
Epoch 3/20, Loss: 2.0841
Epoch 4/20, Loss: 1.8466
Epoch 5/20, Loss: 1.5387
Epoch 6/20, Loss: 1.2775
Epoch 7/20, Loss: 1.2284
Epoch 8/20, Loss: 1.2241
Epoch 9/20, Loss: 1.1384
Epoch 10/20, Loss: 1.0093
Epoch 11/20, Loss: 1.0211
Epoch 12/20, Loss: 0.9718
Epoch 13/20, Loss: 0.8983
Epoch 14/20, Loss: 0.8817
Epoch 15/20, Loss: 0.8691
Epoch 16/20, Loss: 0.8507
Epoch 17/20, Loss: 0.8036
Epoch 18/20, Loss: 0.7769
Epoch 19/20, Loss: 0.7594
Epoch 20/20, Loss: 0.7554


In [82]:
with torch.no_grad():
    outputs = data(X_val_tensor)
    predictions = torch.argmax(outputs, 1)
    correct = predictions == y_val_tensor
    correct_float = correct.float()
    accuracy = correct_float.mean().item()
    
print("Validation accuracy:", accuracy)

Validation accuracy: 0.7544999718666077
